**How to Query the Medicare Dataset (BigQuery)**

In [1]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
medicare = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="cms_medicare")

In [2]:
bq_assistant = BigQueryHelper("bigquery-public-data", "cms_medicare")
bq_assistant.list_tables()

['home_health_agencies_2013',
 'home_health_agencies_2014',
 'hospice_providers_2014',
 'hospital_general_info',
 'inpatient_charges_2011',
 'inpatient_charges_2012',
 'inpatient_charges_2013',
 'inpatient_charges_2014',
 'inpatient_charges_2015',
 'nursing_facilities_2013',
 'nursing_facilities_2014',
 'outpatient_charges_2011',
 'outpatient_charges_2012',
 'outpatient_charges_2013',
 'outpatient_charges_2014',
 'outpatient_charges_2015',
 'part_d_prescriber_2014',
 'physicians_and_other_supplier_2012',
 'physicians_and_other_supplier_2013',
 'physicians_and_other_supplier_2014',
 'physicians_and_other_supplier_2015',
 'referring_durable_medical_equip_2013',
 'referring_durable_medical_equip_2014']

In [3]:
bq_assistant.head("inpatient_charges_2015", num_rows=15)

,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zipcode,drg_definition,hospital_referral_region_description,total_discharges,average_covered_charges,average_total_payments,average_medicare_payments
0,450723,METHODIST CHARLTON MEDICAL CENTER,3500 W WHEATLAND ROAD,DALLAS,TX,75237,309 - CARDIAC ARRHYTHMIA & CONDUCTION DISORDER...,TX - Dallas,36,28753.28,5848.89,4750.42
1,360077,FAIRVIEW HOSPITAL,18101 LORAIN AVENUE,CLEVELAND,OH,44111,"286 - CIRCULATORY DISORDERS EXCEPT AMI, W CARD...",OH - Cleveland,17,54436.24,14316.06,11998.06
2,10056,ST VINCENT'S BIRMINGHAM,810 ST VINCENT'S DRIVE,BIRMINGHAM,AL,35205,377 - G.I. HEMORRHAGE W MCC,AL - Birmingham,25,44590.24,9689.72,7291.76
3,180103,BAPTIST HEALTH LEXINGTON,1740 NICHOLASVILLE ROAD,LEXINGTON,KY,40503,418 - LAPAROSCOPIC CHOLECYSTECTOMY W/O C.D.E. ...,KY - Lexington,18,44002.67,11579.61,10586.50
4,70005,WATERBURY HOSPITAL,64 ROBBINS ST,WATERBURY,CT,6721,439 - DISORDERS OF PANCREAS EXCEPT MALIGNANCY ...,CT - New Haven,13,29188.46,7452.46,6243.38
5,50174,SANTA ROSA MEMORIAL HOSPITAL,1165 MONTGOMERY DR,SANTA ROSA,CA,95405,312 - SYNCOPE & COLLAPSE,CA - Santa Rosa,17,51001.94,6944.18,5699.47
6,210034,MEDSTAR HARBOR HOSPITAL,3001 SOUTH HANOVER STREET,BALTIMORE,MD,21225,"641 - MISC DISORDERS OF NUTRITION,METABOLISM,F...",MD - Baltimore,26,9418.31,8697.42,7639.73
7,60012,CENTURA HEALTH-ST MARY CORWIN MEDICAL CENTER,1008 MINNEQUA AVE,PUEBLO,CO,81004,310 - CARDIAC ARRHYTHMIA & CONDUCTION DISORDER...,CO - Pueblo,12,19345.33,5707.08,2598.67
8,70002,ST FRANCIS HOSPITAL & MEDICAL CENTER,114 WOODLAND STREET,HARTFORD,CT,6105,244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O ...,CT - Hartford,28,46713.50,17270.39,16161.32
9,520193,AURORA BAYCARE MEDICAL CTR,2845 GREENBRIER RD,GREEN BAY,WI,54311,552 - MEDICAL BACK PROBLEMS W/O MCC,WI - Green Bay,13,19185.38,6650.85,4280.46


In [4]:
bq_assistant.table_schema("inpatient_charges_2015")

[SchemaField('provider_id', 'STRING', 'REQUIRED', 'The CMS Certification Number (CCN) of the provider billing for outpatient hospital services', ()),
 SchemaField('provider_name', 'STRING', 'NULLABLE', 'The name of the provider', ()),
 SchemaField('provider_street_address', 'STRING', 'NULLABLE', 'The street address in which the provider is physically located', ()),
 SchemaField('provider_city', 'STRING', 'NULLABLE', 'The city in which the provider is physically located', ()),
 SchemaField('provider_state', 'STRING', 'NULLABLE', 'The state in which the provider is physically located', ()),
 SchemaField('provider_zipcode', 'INTEGER', 'NULLABLE', 'The zip code in which the provider is physically located', ()),
 SchemaField('drg_definition', 'STRING', 'REQUIRED', 'The code and description identifying the MS-DRG. MS-DRGs are a classification system that groups similar clinical conditions (diagnoses) and the procedures furnished by the hospital during the stay', ()),
 SchemaField('hospital_r

What is the total number of medications prescribed in each state?


In [5]:
query1 = """SELECT
  nppes_provider_state AS state,
  ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  state
ORDER BY
  total_claim_count_millions DESC
LIMIT
  5;
        """
response1 = medicare.query_to_pandas_safe(query1)
response1.head(10)

,state,total_claim_count_millions
0,CA,116.0
1,FL,91.0
2,NY,80.0
3,TX,76.0
4,PA,63.0


What is the most prescribed medication in each state?


In [6]:
query2 = """SELECT
  A.state,
  drug_name,
  total_claim_count,
  day_supply,
  ROUND(total_cost_millions) AS total_cost_millions
FROM (
  SELECT
    generic_name AS drug_name,
    nppes_provider_state AS state,
    ROUND(SUM(total_claim_count)) AS total_claim_count,
    ROUND(SUM(total_day_supply)) AS day_supply,
    ROUND(SUM(total_drug_cost)) / 1e6 AS total_cost_millions
  FROM
    `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
  GROUP BY
    state,
    drug_name) A
INNER JOIN (
  SELECT
    state,
    MAX(total_claim_count) AS max_total_claim_count
  FROM (
    SELECT
      nppes_provider_state AS state,
      ROUND(SUM(total_claim_count)) AS total_claim_count
    FROM
      `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
    GROUP BY
      state,
      generic_name)
  GROUP BY
    state) B
ON
  A.state = B.state
  AND A.total_claim_count = B.max_total_claim_count
ORDER BY
  A.total_claim_count DESC
LIMIT
  5;
        """
response2 = medicare.query_to_pandas_safe(query2, max_gb_scanned=10)
response2.head(10)

,state,drug_name,total_claim_count,day_supply,total_cost_millions
0,CA,LEVOTHYROXINE SODIUM,3845087.0,211747380.0,78.0
1,FL,LEVOTHYROXINE SODIUM,2982612.0,163388370.0,64.0
2,TX,HYDROCODONE/ACETAMINOPHEN,2834059.0,60410516.0,63.0
3,NY,AMLODIPINE BESYLATE,2609896.0,123226481.0,21.0
4,PA,LEVOTHYROXINE SODIUM,2353845.0,109168537.0,44.0


What is the average cost for inpatient and outpatient treatment in each city and state?


In [7]:
query3 = """SELECT
  OP.provider_state AS State,
  OP.provider_city AS City,
  OP.provider_id AS Provider_ID,
  ROUND(OP.average_OP_cost) AS Average_OP_Cost,
  ROUND(IP.average_IP_cost) AS Average_IP_Cost,
  ROUND(OP.average_OP_cost + IP.average_IP_cost) AS Combined_Average_Cost
FROM (
  SELECT
    provider_state,
    provider_city,
    provider_id,
    SUM(average_total_payments*outpatient_services)/SUM(outpatient_services) AS average_OP_cost
  FROM
    `bigquery-public-data.cms_medicare.outpatient_charges_2014`
  GROUP BY
    provider_state,
    provider_city,
    provider_id ) AS OP
INNER JOIN (
  SELECT
    provider_state,
    provider_city,
    provider_id,
    SUM(average_medicare_payments*total_discharges)/SUM(total_discharges) AS average_IP_cost
  FROM
    `bigquery-public-data.cms_medicare.inpatient_charges_2014`
  GROUP BY
    provider_state,
    provider_city,
    provider_id ) AS IP
ON
  OP.provider_id = IP.provider_id
  AND OP.provider_state = IP.provider_state
  AND OP.provider_city = IP.provider_city
ORDER BY
  combined_average_cost DESC
LIMIT
  10;
        """
response3 = medicare.query_to_pandas_safe(query3, max_gb_scanned=10)
response3.head(10)

,State,City,Provider_ID,Average_OP_Cost,Average_IP_Cost,Combined_Average_Cost
0,IN,MISHAWAKA,150177,399.0,102521.0,102920.0
1,MI,WARREN,230264,104.0,88620.0,88724.0
2,TX,HOUSTON,450674,88.0,67571.0,67659.0
3,TX,EL PASO,450877,230.0,45179.0,45409.0
4,TN,MEMPHIS,440152,90.0,35698.0,35788.0
5,LA,BATON ROUGE,190128,87.0,34369.0,34456.0
6,LA,LEESVILLE,190297,560.0,32611.0,33172.0
7,NY,VALHALLA,330234,219.0,30975.0,31194.0
8,TX,SAN ANTONIO,670054,585.0,26249.0,26834.0
9,TX,HOUSTON,450289,104.0,26450.0,26553.0


Which are the most common inpatient diagnostic conditions in the United States?

Which cities have the most number of cases for each diagnostic condition?

What are the average payments for these conditions in these cities and how do they compare to the national average?

In [8]:
query4 = """SELECT
  drg_definition AS Diagnosis,
  provider_city AS City,
  provider_state AS State,
  cityrank AS City_Rank,
  CAST(ROUND(citywise_avg_total_payments) AS INT64) AS Citywise_Avg_Payments,
  CONCAT(CAST(ROUND(citywise_avg_total_payments /national_avg_total_payments * 100, 0) AS STRING), " %") AS Avg_Payments_City_vs_National
FROM (
  SELECT
    drg_definition,
    provider_city,
    provider_state,
    cityrank,
    national_num_cases,
    citywise_avg_total_payments,
    national_sum_total_payments,
    (national_sum_total_payments /national_num_cases) AS national_avg_total_payments
  FROM (
    SELECT
      drg_definition,
      provider_city,
      provider_state,
      citywise_avg_total_payments,
      RANK() OVER (PARTITION BY drg_definition ORDER BY citywise_num_cases DESC ) AS cityrank,
      SUM(citywise_num_cases) OVER (PARTITION BY drg_definition ) AS national_num_cases,
      SUM(citywise_sum_total_payments) OVER (PARTITION BY drg_definition ) AS national_sum_total_payments
    FROM (
      SELECT
        drg_definition,
        provider_city,
        provider_state,
        SUM(total_discharges) AS citywise_num_cases,
        SUM(average_total_payments * total_discharges)/ SUM(total_discharges) AS citywise_avg_total_payments,
        SUM(average_total_payments * total_discharges) AS citywise_sum_total_payments
      FROM
        `bigquery-public-data.cms_medicare.inpatient_charges_2014`
      GROUP BY
        drg_definition,
        provider_city,
        provider_state))
  WHERE
    cityrank <=3)  # Limit to top 3 cities for each Diagnosis
ORDER BY
  national_num_cases DESC,
  cityrank
LIMIT
  9;  # Limit Results to the top 3 cities for the top 3 diagnosis;
        """
response4 = medicare.query_to_pandas_safe(query4, max_gb_scanned=10)
response4.head(10)

,Diagnosis,City,State,City_Rank,Citywise_Avg_Payments,Avg_Payments_City_vs_National
0,470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...,NEW YORK,NY,1,21656,144 %
1,470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...,BOSTON,MA,2,18324,122 %
2,470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...,BALTIMORE,MD,3,23056,154 %
3,871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...,HOUSTON,TX,1,14592,106 %
4,871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...,CHICAGO,IL,2,16071,117 %
5,871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...,BROOKLYN,NY,3,23398,170 %
6,291 - HEART FAILURE & SHOCK W MCC,CHICAGO,IL,1,13287,122 %
7,291 - HEART FAILURE & SHOCK W MCC,HOUSTON,TX,2,12572,116 %
8,291 - HEART FAILURE & SHOCK W MCC,BALTIMORE,MD,3,16992,157 %
